In [2]:
!pip install --upgrade pip
!pip install OpenNMT-py
!pip install nltk
!pip install french-lefff-lemmatizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: Could not find a version that satisfies the requirement french-lefff-lemmatizer (from versions: none)
ERROR: No matching distribution found for french-lefff-lemmatizer


In [18]:
%%bash
cat <<EOF > en_fr_train.yaml

## Where the samples will be written
save_data: corpus/run/example
## Where the vocab(s) will be written
src_vocab: corpus/run/train_example.vocab.src
tgt_vocab: corpus/run/train_example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: corpus/Europarl_train_100k.en
        path_tgt: corpus/Europarl_train_100k.fr

    valid:
        path_src: corpus/Europarl_dev_3750.en
        path_tgt: corpus/Europarl_dev_3750.fr



EOF


In [19]:
!onmt_build_vocab -config en_fr_train.yaml -n_sample 10000


Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-05-28 15:46:14,444 INFO] Counter vocab from 10000 samples.
[2024-05-28 15:46:14,444 INFO] Build vocab on 10000 transformed examples/corpus.
[2024-05-28 15:46:14,910 INFO] Counters src: 18628
[2024-05-28 15:46:14,910 INFO] Counters tgt: 23741
[2024-05-28 15:46:14,911 WARNING] path corpus/run/train_example.vocab.src exists, may overwrite...
[2024-05-28 15:46:14,925 WARNING] path corpus/run/train_example.vocab.tgt exists, may overwrite...


In [22]:
%%bash
cat <<EOF > en_fr_train.yaml

## Where the samples will be written
save_data: corpus/run/example
## Where the vocab(s) will be written
src_vocab: corpus/run/train_example.vocab.src
tgt_vocab: corpus/run/train_example.vocab.tgt
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: corpus/Europarl_train_100k.en
        path_tgt: corpus/Europarl_train_100k.fr

    valid:
        path_src: corpus/Europarl_dev_3750.en
        path_tgt: corpus/Europarl_dev_3750.fr

# Model and training parameters
world_size: 1
gpu_ranks: [0]

save_model: corpus/run/model
save_checkpoint_steps: 10000
train_steps: 50000
valid_steps: 10000

EOF


In [23]:
!onmt_train -config en_fr_train.yaml

[2024-05-28 15:48:42,829 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2024-05-28 15:48:42,829 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-05-28 15:48:42,830 INFO] Missing transforms field for valid data, set to default: [].
[2024-05-28 15:48:42,830 INFO] Parsed 2 corpora from -data.
[2024-05-28 15:48:42,830 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2024-05-28 15:48:42,905 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', 'the', 'of', 'to', 'and', 'in', 'is']
[2024-05-28 15:48:42,906 INFO] The decoder start token is: <s>
[2024-05-28 15:48:42,906 INFO] Building model...
[2024-05-28 15:48:43,663 INFO] Switching model to float32 for amp/apex_amp
[2024-05-28 15:48:43,664 INFO] Non quantized layer compute is fp32
[2024-05-28 15:48:43,833 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwi

In [24]:
!onmt_translate -model corpus/run/model_step_50000.pt -src corpus/Europarl_test_500.en -output corpus/pred_run1_indomain.txt -gpu 0 -verbose

[2024-05-28 16:43:12,854 INFO] Loading checkpoint from corpus/run/model_step_50000.pt
[2024-05-28 16:43:13,342 INFO] Loading data into the model
[2024-05-28 16:43:15,818 INFO] 
SENT 1: ['The', 'vote', 'will', 'take', 'place', 'tomorrow', 'at', '<unk>', 'p.m.']
PRED 1: Le vote aura lieu demain à <unk>
PRED SCORE: -0.0028

[2024-05-28 16:43:15,818 INFO] 
SENT 2: ['Financial', 'assistance', 'for', 'Member', "States'", 'balances', 'of', 'payments']
PRED 2: Aide financière des États membres à des <unk> <unk>
PRED SCORE: -0.3220

[2024-05-28 16:43:15,819 INFO] 
SENT 3: ['The', 'next', 'item', 'is', 'the', 'report', '<unk>', 'by', 'Mrs', '<unk>', 'on', 'behalf', 'of', 'the', 'Committee', 'on', 'Economic', 'and', 'Monetary', 'Affairs,', 'on', 'the', 'Commission', 'report', 'to', 'the', 'Council', 'and', 'to', 'the', 'European', 'Parliament', 'on', 'the', 'review', 'of', 'the', 'single', 'facility', 'providing', '<unk>', 'financial', 'assistance', 'for', 'Member', "States'", 'balances', 'of', '

In [25]:
!onmt_translate -model corpus/run/model_step_50000.pt -src corpus/Emea_test_500.en -output corpus/pred_run1_outdomain.txt -gpu 0 -verbose

[2024-05-28 16:43:20,423 INFO] Loading checkpoint from corpus/run/model_step_50000.pt
[2024-05-28 16:43:20,921 INFO] Loading data into the model
[2024-05-28 16:43:22,306 INFO] 
SENT 1: ['<unk>']
PRED 1: <unk>
PRED SCORE: -0.0196

[2024-05-28 16:43:22,306 INFO] 
SENT 2: ['<unk>', '<unk>', '<unk>']
PRED 2: <unk> <unk> <unk>
PRED SCORE: -0.0850

[2024-05-28 16:43:22,306 INFO] 
SENT 3: ['16.']
PRED 3: <unk>
PRED SCORE: -0.0540

[2024-05-28 16:43:22,306 INFO] 
SENT 4: ['<unk>', '<unk>', '<unk>']
PRED 4: <unk> <unk> <unk>
PRED SCORE: -0.0850

[2024-05-28 16:43:22,306 INFO] 
SENT 5: ['<unk>', '4000', '<unk>', '<unk>', '<unk>']
PRED 5: Rapport <unk> <unk> <unk> <unk> <unk>
PRED SCORE: -0.1114

[2024-05-28 16:43:22,307 INFO] 
SENT 6: ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
PRED 6: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
PRED SCORE: -0.0546

[2024-05-28 16:43:22,307 INFO] 
SENT 7: ['<unk>']
PRED 7: <unk>
PRED SCORE: -0.0196

[2024-05-28 16:

In [26]:
!pip install gensim
!pip install sacrebleu

In [28]:
from sacrebleu import corpus_bleu


with open("corpus/pred_run2.txt") as pred_file:
    pred_lines = pred_file.readlines()

with open("corpus/Europarl_test_500.fr") as ref_file:
    ref_lines = ref_file.readlines()


bleu = corpus_bleu(pred_lines, [ref_lines])
print("BLEU score: ", bleu.score)


BLEU score:  20.035572738611865


In [29]:
%%shell
sacrebleu corpus/Europarl_test_500.fr -i corpus/pred_run1_indomain.txt -m bleu
sacrebleu corpus/Emea_test_500.fr -i corpus/pred_run1_outdomain.txt -m bleu

{
 "name": "BLEU",
 "score": 19.0,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.2",
 "verbose_score": "42.8/23.2/14.3/9.1 (BP = 1.000 ratio = 1.050 hyp_len = 16628 ref_len = 15832)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.2"
}
{
 "name": "BLEU",
 "score": 0.2,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.2",
 "verbose_score": "5.6/2.2/0.0/0.0 (BP = 1.000 ratio = 1.974 hyp_len = 5605 ref_len = 2839)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.2"
}
